In [3]:
f = open('/Users/jackweissenberger/Documents/human-eval/data/HumanEval.jsonl', 'r')
file = f.readlines()
f.close()

In [4]:
import json

In [5]:
json.loads(file[0])

{'task_id': 'HumanEval/0',
 'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n',
 'entry_point': 'has_close_elements',
 'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n',
 'test': "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9,

In [6]:
q1 = json.loads(file[1])

In [8]:
print(q1['test'])



METADATA = {
    'author': 'jt',
    'dataset': 'test'
}


def check(candidate):
    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True
    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False
    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True
    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False
    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True
    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True
    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False




In [14]:
print(q1['prompt'])

from typing import List


def separate_paren_groups(paren_string: str) -> List[str]:
    """ Input to this function is a string containing multiple groups of nested parentheses. Your goal is to
    separate those group into separate strings and return the list of those.
    Separate groups are balanced (each open brace is properly closed) and not nested within each other
    Ignore any spaces in the input string.
    >>> separate_paren_groups('( ) (( )) (( )( ))')
    ['()', '(())', '(()())']
    """



In [12]:
import  requests
def find_examples(docstrings: str, num_examples: int=10) -> list:
    """
    Searches example database for most relevant examples to aid in code generation
    """
    # currently hit local running bm25 database
    url = "http://127.0.0.1:8000/get_answers"
    data = {"query": docstrings, "n_answers": num_examples}
    r = requests.post(url=url, json=data)
    answers = r.json()["answers"]
    return answers

In [16]:
from execution import check_correctness

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

/Users/jackweissenberger/anaconda3/envs/spec-trans/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("../codegen/350-mono-tokenizer")
model = AutoModelForCausalLM.from_pretrained("../codegen/350-mono-model")

Some weights of the model checkpoint at ../codegen/350-mono-model were not used when initializing CodeGenForCausalLM: ['transformer.h.13.attn.bias', 'transformer.h.15.attn.bias', 'transformer.h.16.attn.bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.12.attn.bias', 'transformer.h.19.attn.bias', 'transformer.h.14.attn.bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.bias', 'transformer.h.6.attn.bias', 'transformer.h.3.attn.bias', 'transformer.h.11.attn.bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.5.attn.bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.16.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.12.attn.masked_bias', 'transformer.h.15.attn.masked_bias', 'transformer.h.14.attn.masked_bias', 'transformer.h.18.attn.masked_bias', 'transfo

In [108]:
from transformers.generation_stopping_criteria import StoppingCriteria
import torch
class CodeOnlyWithinFunction(StoppingCriteria):
    """
    This is the stopping criteria for code generation models such that they only generate code
    that starts each line with a tab. This restrics the models from generating code outside
    of a function.
    """

    def __init__(self):
        # newline: 198
        # tab: 197
        # "    ": 50284
        # 8 spaces: 50280
        self.ok_tokens = {198, 197, 50284, 50280, 220}

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        """
        Args:
            input_ids (`torch.LongTensor` of shape `(batch_size, sequence_length)`):
                Indices of input sequence tokens in the vocabulary.

                Indices can be obtained using [`BertTokenizer`]. See [`PreTrainedTokenizer.encode`] and
                [`PreTrainedTokenizer.__call__`] for details.

                [What are input IDs?](../glossary#input-ids)
            scores (`torch.FloatTensor` of shape `(batch_size, config.vocab_size)`):
                Prediction scores of a language modeling head. These can be scores for each vocabulary token before SoftMax
                or scores for each vocabulary token after SoftMax.
            kwargs:
                Additional stopping criteria specific kwargs.

        Return:
            `bool`. `False` indicates we should continue, `True` indicates we should stop.
        """

        # decode
        splits = tokenizer.decode(input_ids[0]).split('\n')

        for i in reversed(range(len(splits))):
            if splits[i] == '' or len(splits[i]) == 0:
                continue
            if splits[i].startswith('def '):
                # this is the case where we have gotten back to the function defintion and can allow lines without tabs
                return False
            if not splits[i][0].isspace():
                # stop generation
                return True
        
        return False

In [109]:
cf = CodeOnlyWithinFunction()
cf(tokenizer(q1["prompt"], return_tensors="pt").input_ids, scores=None)

False

In [111]:
def model_inference(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    generated_ids = model.generate(input_ids, max_length=1024, temperature=0.2, stopping_criteria=[cf], pad_token_id=2)
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [112]:
model_output = model_inference(q1["prompt"])

In [113]:
print(model_output)

from typing import List


def separate_paren_groups(paren_string: str) -> List[str]:
    """ Input to this function is a string containing multiple groups of nested parentheses. Your goal is to
    separate those group into separate strings and return the list of those.
    Separate groups are balanced (each open brace is properly closed) and not nested within each other
    Ignore any spaces in the input string.
    >>> separate_paren_groups('( ) (( )) (( )( ))')
    ['()', '(())', '(()())']
    """
    paren_groups = []
    for i in range(len(paren_string)):
        if paren_string[i] == '(':
            paren_groups.append(paren_string[i:])
        elif paren_string[i] == ')':
            if len(paren_groups) > 0:
                paren_groups.pop()
            else:
                return []
    return paren_groups


def


In [143]:
def clean_up_model_output(model_output):
    splits = model_output.split('\n')
    for i in reversed(range(len(splits))):
        if splits[i] == '' or len(splits[i]) == 0:
            continue
        if splits[i][0].isspace():
            splits = splits[:i+1]
            break
    return '\n'.join(splits)


In [144]:
cleanded_output = clean_up_model_output(model_output)

In [146]:
print(cleanded_output)

from typing import List


def separate_paren_groups(paren_string: str) -> List[str]:
    """ Input to this function is a string containing multiple groups of nested parentheses. Your goal is to
    separate those group into separate strings and return the list of those.
    Separate groups are balanced (each open brace is properly closed) and not nested within each other
    Ignore any spaces in the input string.
    >>> separate_paren_groups('( ) (( )) (( )( ))')
    ['()', '(())', '(()())']
    """
    paren_groups = []
    for i in range(len(paren_string)):
        if paren_string[i] == '(':
            paren_groups.append(paren_string[i:])
        elif paren_string[i] == ')':
            if len(paren_groups) > 0:
                paren_groups.pop()
            else:
                return []
    return paren_groups


In [149]:
print(q1['test'])



METADATA = {
    'author': 'jt',
    'dataset': 'test'
}


def check(candidate):
    assert candidate('(()()) ((())) () ((())()())') == [
        '(()())', '((()))', '()', '((())()())'
    ]
    assert candidate('() (()) ((())) (((())))') == [
        '()', '(())', '((()))', '(((())))'
    ]
    assert candidate('(()(())((())))') == [
        '(()(())((())))'
    ]
    assert candidate('( ) (( )) (( )( ))') == ['()', '(())', '(()())']

